# PROJET WEB SCRAPING ISE 3

#### Ce projet a pour but d'évaluer vos acquis lors du cours et de mettre en valeur les compétences acquises.

Vous êtes un Data Scientist embauché dans une grande entreprise de la place qui oeuvre dans le domaine de la publication des films. Vous avez besoin de certaines données sur les critiques des films du site **senscritique** afin de vérifier à une fréquence journalière, les données mises sur ce site.

I - Votre première mission est d'accéder aux films https://www.senscritique.com/films et d'extraire :

    1- Les films du moment ainsi que leurs notes
    2- Les sortis de la semaine ainsi que leurs notes
    3- A voir en streaming ainsi que leurs notes
    
    Mettre toutes ces données dans un dataframe et exporter le en xlsx.

II - Ensuite, votre seconde mission est d'accéder aux critiques populaires et aux réactions populaires. Pour ceux-ci, vous devez extraire pour chaque film critiqué :

    1 - Le nom de celui qui a fait la critique ;
    2 - Le titre de la critique ;
    3 - La note attribuée par ce dernier ;
    4 - La date de la critique ;
    5 - Le nombre de "j'aime" ;
    6 - Le nombre de commentaires
    7 - La photo de profil de l'auteur de la critique
    8 - Le contenu de la critique
        
     Mettre toutes ces données dans un dataframe et exporter le en xlsx.
        
**III - Enfin, à l'aide des cours vus dans d'autres modules, faites une analyse de sentiments sur les critiques faites et les réactions.**

<p style='color:darkred'><strong><i>NB : Faites une capture d'écran des Conditions Générales d'Utilisation (CGU) permettant de valider ou pas le fait de scrapper des données sur ce site.</strong></i></p>


<p style='color:darkorange'><strong>INSTRUCTIONS</strong></p>

Votre script doit s'exécuter chaque jour à une heure aléatoire entre **08h et 10H.**

Ce travail est à faire individuellement.

Vous devez répondre à toutes les questions dans un seul notebook et vous assurez que vous avez pris en compte toutes les bonnes pratiques vues au cours. Le code doit être enregistré sous le format Code_Nom_Prenoms_ISE3_options.py

Les bases de données extraites doivent être sous format Excel Base_Nom_Prenoms_ISE3_mission1.xlsx & Base_Nom_Prenoms_ISE3_mission2.xlsx.

La capture des Conditions Générales d'Utilisation CGU_Nom_Prenoms_ISE3_options.png et doit pouvoir être affichée dans le notebook.

Le tout zippé sous un dossier nommé Nom_Prenoms.

Le dossier doit être envoyé à l'adresse email : **dackounelie@gmail.com** au plus tard **Vendredi 17 Mai 2024 à 22H.**

#### Bon projet à Tous !!!!

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests import get
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

In [3]:
# Vérification du code status
response = requests.get('https://www.senscritique.com/films')
status_code = response.status_code
status_code

200

In [4]:
url = 'https://www.senscritique.com/films'
response = get(url)
print(response.text[:1200])

<!DOCTYPE html><html lang="fr"><head><meta charSet="utf-8" /><meta name="viewport" content="width=device-width" /><meta name="robots" content="index,follow,max-snippet:-1" /><meta name="author" content="SensCritique" /><meta name="country" content="France" /><meta name="geo.position" content="48.8685823;2.3480648" /><meta name="geo.country" content="FR" /><meta name="ICBM" content="48.8685823,2.3480648" /><meta property="fb:pages" content="127524802986" /><link href="/app-icons/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57" /><link href="/app-icons/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60" /><link href="/app-icons/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72" /><link href="/app-icons/apple-touch-icon-76x76.png" rel="apple-touch-icon" sizes="76x76" /><link href="/app-icons/apple-touch-icon-114x114.png" rel="apple-touch-icon" sizes="114x114" /><link href="/app-icons/apple-touch-icon-120x120.png" rel="apple-touch-icon" sizes="120

# MISSION 1

## Films du moment

In [5]:
url1 = "https://www.senscritique.com/films/toujours-a-l-affiche"
response = get(url1)
page1 = response.content
soup = BeautifulSoup(page1, 'html.parser')
all_films_moment = soup.find_all(class_ ="sc-f52bf7ce-0 drGGMe sc-28c72235-1 eVgUeC")

In [6]:
noms_films_moment = []
note_films_moment = []

for element in all_films_moment:
    if element.find(class_ = "sc-8251ce8c-5 fTXQip") != None:
        rate = float(element.find(class_ = "sc-8251ce8c-5 fTXQip").text)
        note_films_moment.append(rate)
    else:
        note_films_moment.append(np.nan)
    nom = element.find(class_ = "sc-e6f263fc-0 sc-df6b780a-1 cTWuvs lbhoSA").text
    noms_films_moment.append(nom)

## Sorties de la semaine

In [7]:
url2 = "https://www.senscritique.com/films/cette-semaine"
response = get(url2)
page2 = response.content
soup2 = BeautifulSoup(page2, 'html.parser')

In [10]:
script_tag = soup2.find('script', text=re.compile(r'__APOLLO_STATE__'))
script_content = script_tag.string
groupe = "__APOLLO_STATE__"
pattern = re.escape(groupe) + r'(.*)'
match = re.search(pattern, script_content)
result = match.group(1).strip()
nom_sorties_de_la_semaine = re.findall(r'"title":"(.*?)"', result)
note_sorties_de_la_semaine = re.findall(r'"rating":(.*?)(?:,|})', result)

C:\Users\yanno\AppData\Local\Temp\ipykernel_4592\3774709640.py:1: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  script_tag = soup2.find('script', text=re.compile(r'__APOLLO_STATE__'))


## A voir en streaming

In [11]:
url3 = "https://www.senscritique.com/films/streaming"
response = get(url3)
page3 = response.content
soup = BeautifulSoup(page3, 'html.parser')
all_films_a_voir = soup.find_all(class_ ="sc-ee95228d-5 enEfMN")

In [12]:
noms_films_a_voir = []
note_films_a_voir = []

for element in all_films_a_voir:
    if element.find(class_ = "sc-8251ce8c-5 bVyLNx globalRating") != None:
        rate = float(element.find(class_ = "sc-8251ce8c-5 bVyLNx globalRating").text)
        note_films_a_voir.append(rate)
    else:
        note_films_a_voir.append(np.nan)
    nom = element.find(class_ = "sc-e6f263fc-0 sc-ee95228d-1 GItpw gJUtFN").text
    noms_films_a_voir.append(nom)

In [13]:
# Créer du dataframe contenant les données des films
films_data = {
    'Films du moment': {
        'Nom du film': noms_films_moment,
        'Note': note_films_moment
    },
    'Sorties de la semaine': {
        'Nom du film': nom_sorties_de_la_semaine,
        'Note': note_sorties_de_la_semaine
    },
    'Films à voir en streaming': {
        'Nom du film': noms_films_a_voir,
        'Note': note_films_a_voir
    }
}

# Créer une liste de dictionnaires pour chaque catégorie de films
category_dicts = []
for category, films in films_data.items():
    category_dict = {'Catégorie': [category] * len(films['Nom du film']),
                     'Nom du film': films['Nom du film'],
                     'Note': films['Note']}
    category_dicts.append(category_dict)

# Concaténer les dictionnaires en un seul DataFrame
df_films = pd.concat([pd.DataFrame(category_dict) for category_dict in category_dicts], ignore_index=True)

# Afficher le DataFrame
df_films

,Catégorie,Nom du film,Note
0,Films du moment,Le Deuxième Acte,6.3
1,Films du moment,Challengers,6.6
2,Films du moment,La Planète des singes - Le Nouveau Royaume,6.5
3,Films du moment,Un p'tit truc en plus,6.8
4,Films du moment,The Fall Guy,6.5
...,...,...,...
63,Films à voir en streaming,Adagio,6.9
64,Films à voir en streaming,Unfrosted - L'épopée de la Pop-Tart,4.5
65,Films à voir en streaming,Baghead,4.9
66,Films à voir en streaming,Baby Ruby,5.4


# Exportation du DataFrame dans un fichier Excel

In [14]:
with pd.ExcelWriter('Base_ARRA_Othniel_ISE3_mission1.xlsx') as writer:
    df_films.to_excel(writer, sheet_name='Films_et_leurs_notes', index=False)

# MISSION 2

In [15]:
url = 'https://www.senscritique.com/films'
response = get(url)
page = response.content
soup = BeautifulSoup(page, 'html.parser')
all_comments = soup.find_all(class_ ="sc-328d61f-0 fARllz sc-d36dcab1-0 hOLUBG")
all_reactions = soup.find_all(class_="sc-328d61f-0 fARllz sc-def8c7bf-0 kEKJHJ")

In [16]:
noms_des_films_critiques = []
noms_des_critiqueurs = []
titre_critique = []
note_attribuee = []
date_critique = []
nbre_jaime = []
nombre_commentaire = []
photo_de_profil = []
contenu_critique = []

noms_des_films_r = []
noms_des_reacteurs = []
titre_reaction = []
date_reaction = []
nbre_jaime2 = []
nombre_commentaire2 = []
photo_de_profil2 = []
contenu_reaction = []

# Critiques populaires
section_0 = all_comments[0].find_all(class_ = "sc-328d61f-4 fdFAPT")
for element in section_0:
    nom_films = element.find(class_ = "sc-e6f263fc-0 sc-df6b780a-1 iZcnfG lbhoSA").text
    noms_des_films_critiques.append(nom_films)
    critiqueurs = element.find(class_ = "sc-e6f263fc-0 sc-b7da4c5c-2 GItpw gvGKDY").text
    noms_des_critiqueurs.append(critiqueurs) 
    titres = element.find(class_ ="sc-e6f263fc-0 sc-4db36029-4 bTynbZ irayCk").text
    titre_critique.append(titres)
    notes = element.find(class_ = "sc-8251ce8c-3 gbAIun sc-4db36029-3 iepouY").text
    note_attribuee.append(notes)
    dates = element.find(class_ = "sc-e6f263fc-0 sc-3b5d9ff1-0 GItpw dOiIbg").text
    date_critique.append(dates)
    if element.find(class_ = "sc-e6f263fc-0 sc-8d1083e0-1 GItpw XTOao") != None:
        jaimes = element.find(class_ = "sc-e6f263fc-0 sc-8d1083e0-1 GItpw XTOao").text
        nbre_jaime.append(jaimes)
    else:
        nbre_jaime.append("0 j'aime")
    if element.find(class_ = "sc-e6f263fc-0 sc-3fcb6f61-1 GItpw djcKen") != None:
        commentaires = element.find(class_ = "sc-e6f263fc-0 sc-3fcb6f61-1 GItpw djcKen").text
        nombre_commentaire.append(commentaires)
    else:
        nombre_commentaire.append(0)
    photos = element.find("div", class_ = "sc-1a97d7be-0 gFHNhq")
    photos = photos.find("a", class_ = "sc-1a97d7be-3 fZBrFn")
    if photos != None:
        photo_links = photos.get("data-srcname")
        photo_de_profil.append(photo_links)
    else:
        photo_de_profil.append(np.nan)
    critiques = element.find(class_ = "sc-e6f263fc-2 ewlOkV").text
    contenu_critique.append(critiques)


    
# Réactions populaires
section_1 = all_reactions[0].find_all(class_ = "sc-328d61f-4 fdFAPT")
for element in section_1:
    nom_films = element.find(class_ = "sc-e6f263fc-0 iZcnfI").text
    noms_des_films_r.append(nom_films)
    dates = element.find(class_ = "sc-e6f263fc-0 sc-3b5d9ff1-0 GItpw dOiIbg").text
    date_reaction.append(dates)
    if element.find(class_ = "sc-e6f263fc-0 sc-8d1083e0-1 GItpw XTOao") != None:
        jaimes = element.find(class_ = "sc-e6f263fc-0 sc-8d1083e0-1 GItpw XTOao").text
        nbre_jaime2.append(jaimes)
    else:
        nbre_jaime2.append("0 j'aime")
    if element.find(class_ = "sc-e6f263fc-0 sc-3fcb6f61-1 GItpw djcKen") != None:
        commentaires = element.find(class_ = "sc-e6f263fc-0 sc-3fcb6f61-1 GItpw djcKen").text
        nombre_commentaire2.append(commentaires)
    else:
        nombre_commentaire2.append(0)
    critiques = element.find(class_ = "sc-1e6d9d58-0 dnxDDB sc-6687e0c2-6 bByzmL").text
    contenu_reaction.append(critiques)
    
    
df_critiques = pd.DataFrame(
    {'Films critiqués': noms_des_films_critiques, 
     'Critiqueurs': noms_des_critiqueurs,
     'Titre des critiques' : titre_critique,
     'Note attribuée' : note_attribuee,
     'Date de la critique' : date_critique,
     'Nombre de j''aime' : nbre_jaime,
     'Nombre de commentaire' : nombre_commentaire,
     'Photo' : photo_de_profil,
     'Contenu des critiques' : contenu_critique
    }
)

df_reactions = pd.DataFrame(
    {'Films ayant des réactions': noms_des_films_r, 
     'Date de la critique' : date_reaction,
     'Nombre de j''aime' : nbre_jaime2,
     'Nombre de commentaire' : nombre_commentaire2,
     #'Photo' : photo_de_profil2,
     'Contenu des critiques' : contenu_reaction
    }
)


# Exportation des DataFrames dans un fichier Excel

In [17]:
with pd.ExcelWriter('Base_ARRA_Othniel_ISE3_mission2.xlsx') as writer:
    df_critiques.to_excel(writer, sheet_name='Critiques', index=False)
    df_reactions.to_excel(writer, sheet_name='Reactions', index=False)

# Analyse des sentiments

In [18]:
def preprocess_text(text):
    text = text.lower()
    words = word_tokenize(text)
    words = [word for word in words if word.isalnum() and word not in stopwords.words('french')]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    preprocessed_text = ' '.join(words)
    return preprocessed_text

In [19]:
# Prétraitement des critiques et réactions
preprocessed_critiques = [preprocess_text(critique) for critique in contenu_critique]
preprocessed_reactions = [preprocess_text(reaction) for reaction in contenu_reaction]

In [20]:
# Analyse de sentiment
sia = SentimentIntensityAnalyzer()

def analyse_sentiment(text):
    scores = sia.polarity_scores(text)
    if scores['compound'] >= 0.05:
        return 'Positif'
    elif scores['compound'] <= -0.05:
        return 'Negatif'
    else:
        return 'Neutre'

# Analyse de sentiment des critiques
sentiments_critiques = [analyse_sentiment(critique) for critique in preprocessed_critiques]

# Analyse de sentiment des réactions
sentiments_reactions = [analyse_sentiment(reaction) for reaction in preprocessed_reactions]

# Création des DataFrames
df_critiques = pd.DataFrame({
    'Critiques': contenu_critique,
    'Sentiment': sentiments_critiques
})

df_reactions = pd.DataFrame({
    'Réactions': contenu_reaction,
    'Sentiment': sentiments_reactions
})

# Affichage des résultats
print("### Critiques ###")
print(df_critiques)
print("\n### Réactions ###")
print(df_reactions)

### Critiques ###
                                           Critiques Sentiment
0  Je vous demande pardon d'avoir décroché cette ...   Positif
1  On pouvait s’étonner de l’absence de Dupieux à...    Neutre
2  Celles et ceux qui sont des fans du cinéma du ...   Positif

### Réactions ###
                                           Réactions Sentiment
0  On s'en doutait, les critiques cannoises se dé...   Negatif
1  Où le peuple SC commence à se rendre compte de...    Neutre
2  Sortie par Metropolitan le 3 juillet. 2ème par...    Neutre
